In [1]:

%pprint
import sys
sys.path.insert(1, '../py')

Pretty printing has been turned OFF


In [2]:

import os
from pandas import DataFrame
import pandas as pd
from FRVRS import nu
import numpy as np

In [3]:

# Get all CSVs into one data frame
if nu.pickle_exists('frvrs_logs_df'):
    frvrs_logs_df = nu.load_object('frvrs_logs_df')
    print(frvrs_logs_df.shape)
    # df = frvrs_logs_df.sample(4).dropna(axis='columns', how='all')
    # display(df.T)

(832366, 109)



# Number of pulses taken by number of patients treated

In [5]:

# Create a mask to filter rows where action_type is either 'PULSE_TAKEN' or 'INJURY_TREATED'
mask_series = frvrs_logs_df.action_type.isin(['PULSE_TAKEN', 'INJURY_TREATED'])

# Define the columns to group by


# Get the scene with the most pulses taken
values_list = frvrs_logs_df[mask_series].groupby(fu.patient_groupby_columns).size().sort_values().tail(1).index.item()

# Initialize a base mask series as True
base_mask_series = True

# Update the base mask series with the grouping columns
for cn, cv in zip(fu.patient_groupby_columns, values_list): base_mask_series &= (frvrs_logs_df[cn] == cv)

# Combine the base mask series with a mask for 'PULSE_TAKEN' actions
mask_series = base_mask_series & frvrs_logs_df.action_type.isin(['PULSE_TAKEN'])

# Count the number of PULSE_TAKEN actions in the filtered dataframe
pulses_count = frvrs_logs_df[mask_series].shape[0]

# Print the number of pulses taken to treat one patient
print(f'This responder took {pulses_count} pulses to treat one patient.')

This responder took 18 pulses to treat one patient.


In [5]:

# Number of pulses taken by number of patients treated
if nu.pickle_exists('pulses_count_df'):
    pulses_count_df = nu.load_object('pulses_count_df')
else:
    
    # Create a boolean mask to filter rows with action_type as PULSE_TAKEN or INJURY_TREATED
    mask_series = frvrs_logs_df.action_type.isin(['PULSE_TAKEN', 'INJURY_TREATED'])
    
    # Define the columns to group by
    
    
    # Initialize lists to store rows and indices
    rows_list = []
    indices_list = []
    
    # Iterate through groups defined by the columns
    for (session_uuid, scene_id, patient_id), patient_df in frvrs_logs_df[mask_series].groupby([fu.patient_groupby_columns]):
        
        # Create a tuple for the current group's index
        index_tuple = (session_uuid, scene_id, patient_id)
        indices_list.append(index_tuple)
        
        # Create a dictionary to store row data
        row_dict = {}
        
        # Get the logger version for the current group
        row_dict['logger_version'] = fu.get_logger_version(patient_df)
        
        # Create a mask for 'PULSE_TAKEN' actions and count the occurrences
        row_dict['pulses_count'] = fu.get_pulse_taken_count(patient_df)
        
        # Create a mask for 'INJURY_TREATED' actions and count the occurrences
        row_dict['treated_count'] = fu.get_injury_treated_count(patient_df)
        
        # Try to calculate the average number of pulses taken per injury treated for the current group
        try: row_dict['pulses_by_treated'] = row_dict['pulses_count'] / row_dict['treated_count']
        
        # If there are no injuries treated, set the value to NaN
        except ZeroDivisionError: row_dict['pulses_by_treated'] = np.nan
        
        # Append the row dictionary to the list of rows
        rows_list.append(row_dict)
    
    # Create a DataFrame with rows and multi-level index
    pulses_count_df = DataFrame(rows_list, index=pd.MultiIndex.from_tuples(tuples=indices_list, names=fu.patient_groupby_columns))
    nu.store_objects(pulses_count_df=pulses_count_df)

Pickling to C:\Users\DaveBabbitt\Documents\GitHub\itm-analysis-reporting\saves\pkl\pulses_count_df.pkl


In [12]:

# Number of pulses taken by number of patients treated for the scene
df = pulses_count_df.reset_index(drop=False).groupby(fu.scene_groupby_columns).filter(
    lambda df: not any(df.treated_count == 0)
).groupby(fu.scene_groupby_columns).sum()

# Try to calculate the average number of pulses taken per injury treated for the current scene
try:
    df['pulses_by_treated'] = df['pulses_count'] / df['treated_count']

# If there are no injuries treated, set the value to NaN
except ZeroDivisionError:
    df['pulses_by_treated'] = np.nan

ave_pulses_by_treated = df.pulses_by_treated.mean()
print(f'The average number of pulses taken per injury treated per scene is {ave_pulses_by_treated:0.2}.')

The average number of pulses taken per injury treated per scene is 2.0.


In [8]:

# Group the 'pulses_count_df' DataFrame by the scene columns and calculate the sum
df = pulses_count_df.groupby(fu.scene_groupby_columns).sum()

# Select only the 'pulses_count' and 'treated_count' columns from the grouped DataFrame
columns_list = ['pulses_count', 'treated_count']
df = df[columns_list]

# Try to calculate the 'pulses_by_treated' column by dividing 'pulses_count' by 'treated_count'
try: df['pulses_by_treated'] = df['pulses_count'] / df['treated_count']

# Handle the case of ZeroDivisionError by assigning 'np.nan' to the 'pulses_by_treated' column
except ZeroDivisionError:
    df['pulses_by_treated'] = np.nan

# Create a boolean mask to identify rows where 'pulses_by_treated' is null
mask_series = pulses_count_df.pulses_by_treated.isnull()

# Define a list of columns to be displayed from the filtered DataFrame
columns_list = ['pulses_count', 'treated_count', 'pulses_by_treated']

# Display the specified columns for rows where 'pulses_by_treated' is null
display(pulses_count_df[mask_series][columns_list])

# Create a mask to filter rows in the data frame based on whether their index is in a specific condition
mask_series = df.index.isin([(s, t) for (s, t), df in pulses_count_df[mask_series].groupby(fu.scene_groupby_columns)])

# Apply the mask to the 'df' DataFrame to select the desired rows
df[mask_series]

pulses_count  \
session_uuid                         scene_id patient_id                                              
04882754-51a0-4e5b-9617-b200e3609116 2          Bob_0 Root                                          2   
                                                Gary_1 Root                                         1   
06574b6f-ab02-432c-9a65-7b031218a270 2          Bob_0 Root                                          1   
                                                Gary_1 Root                                         1   
                                                Gary_9 Root                                         1   
...                                                                                               ...   
ff4c3532-5924-45b6-9a6a-d0d3274b6ef0 2          Intelligence Officer Burned_Gary_1 Root             2   
                                                Military Mike Jungle Scout_1_3 Root                 1   
ff97fc56-c803-4aa7-b419-35e0ac17d0b3 2          Gary_1 Root                                         3   
                                                Gary_9 Root                                         1   
                                                Gloria_6 Root                                       1   

                                                                                         treated_count  \
session_uuid                         scene_id patient_id                                               
04882754-51a0-4e5b-9617-b200e3609116 2          Bob_0 Root                                           0   
                                                Gary_1 Root                                          0   
06574b6f-ab02-432c-9a65-7b031218a270 2          Bob_0 Root                                           0   
                                                Gary_1 Root                                          0   
                                                Gary_9 Root                                          0   
...                                                                                                ...   
ff4c3532-5924-45b6-9a6a-d0d3274b6ef0 2          Intelligence Officer Burned_Gary_1 Root              0   
                                                Military Mike Jungle Scout_1_3 Root                  0   
ff97fc56-c803-4aa7-b419-35e0ac17d0b3 2          Gary_1 Root                                          0   
                                                Gary_9 Root                                          0   
                                                Gloria_6 Root                                        0   

                                                                                         pulses_by_treated  
session_uuid                         scene_id patient_id                                                  
04882754-51a0-4e5b-9617-b200e3609116 2          Bob_0 Root                                             NaN  
                                                Gary_1 Root                                            NaN  
06574b6f-ab02-432c-9a65-7b031218a270 2          Bob_0 Root                                             NaN  
                                                Gary_1 Root                                            NaN  
                                                Gary_9 Root                                            NaN  
...                                                                                                    ...  
ff4c3532-5924-45b6-9a6a-d0d3274b6ef0 2          Intelligence Officer Burned_Gary_1 Root                NaN  
                                                Military Mike Jungle Scout_1_3 Root                    NaN  
ff97fc56-c803-4aa7-b419-35e0ac17d0b3 2          Gary_1 Root                                            NaN  
                                                Gary_9 Root                                            NaN  
                                          

pulses_count  treated_count  \
session_uuid                         scene_id                                
04882754-51a0-4e5b-9617-b200e3609116 2                     14              9   
06574b6f-ab02-432c-9a65-7b031218a270 2                     22              9   
                                     5                     30              8   
                                     7                     14             10   
                                     10                    15             11   
...                                                       ...            ...   
f8b5cbd1-0abe-483b-8b16-956044dc0119 1                     11              2   
f90a6408-2a18-49b4-8a6b-edee48e7e9ed 2                     18              9   
fde2c201-36fc-4123-bf42-f1bff1b24969 1                      8              2   
ff4c3532-5924-45b6-9a6a-d0d3274b6ef0 2                      7              2   
ff97fc56-c803-4aa7-b419-35e0ac17d0b3 2                     20              7   

                                                 pulses_by_treated  
session_uuid                         scene_id                     
04882754-51a0-4e5b-9617-b200e3609116 2                    1.555556  
06574b6f-ab02-432c-9a65-7b031218a270 2                    2.444444  
                                     5                    3.750000  
                                     7                    1.400000  
                                     10                   1.363636  
...                                                            ...  
f8b5cbd1-0abe-483b-8b16-956044dc0119 1                    5.500000  
f90a6408-2a18-49b4-8a6b-edee48e7e9ed 2                    2.000000  
fde2c201-36fc-4123-bf42-f1bff1b24969 1                    4.000000  
ff4c3532-5924-45b6-9a6a-d0d3274b6ef0 2                    3.500000  
ff97fc56-c803-4aa7-b419-35e0ac17d0b3 2                    2.857143  

[142 rows x 3 columns]